### Before you start: don't forget to change your runtime

You might need to change the run time to "GPU" in order to run your code successfully (under Runtime -> Change runtime type).

### Code Challenge Question

Using the `pubmed_dataset_student.h5` that has been been randomly split into train, val and test, modify the existing `GAT` archicture to get 80% accuracy. Bonus points for anyone that manages to get over 80% accuracy! 

HINT: Experiment with different types of activation functions, neighbourhood sizes and convolutional layers. 

# Installation

In [1]:
!pip uninstall -y torch torchvision torchtext fastai
!pip install torch==1.6.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html

!pip install torch-scatter==latest+cu101 torch-sparse==latest+cu101 -f https://s3.eu-central-1.amazonaws.com/pytorch-geometric.com/whl/torch-1.6.0.html
!pip install torch-spline-con==latest+cu101 torch-scatter==latest+cu101 -f https://s3.eu-central-1.amazonaws.com/pytorch-geometric.com/whl/torch-1.6.0.html
!pip install torch-sparse==latest+cu101 -f https://s3.eu-central-1.amazonaws.com/pytorch-geometric.com/whl/torch-1.6.0.html

!pip install torch-geometric==1.6.1

!pip install git+https://github.com/Aggregate-Intellect/tutorial-notebook-utils.git

Uninstalling torch-1.6.0+cu101:
  Successfully uninstalled torch-1.6.0+cu101
Looking in links: https://download.pytorch.org/whl/torch_stable.html
  Using cached https://download.pytorch.org/whl/cu101/torch-1.6.0%2Bcu101-cp37-cp37m-linux_x86_64.whl
Looking in links: https://s3.eu-central-1.amazonaws.com/pytorch-geometric.com/whl/torch-1.6.0.html
  Using cached https://s3.eu-central-1.amazonaws.com/pytorch-geometric.com/whl/torch-1.6.0/torch_scatter-latest%2Bcu101-cp37-cp37m-linux_x86_64.whl
  Using cached https://s3.eu-central-1.amazonaws.com/pytorch-geometric.com/whl/torch-1.6.0/torch_sparse-latest%2Bcu101-cp37-cp37m-linux_x86_64.whl
  Found existing installation: torch-scatter 2.0.5
    Uninstalling torch-scatter-2.0.5:
      Successfully uninstalled torch-scatter-2.0.5
  Found existing installation: torch-sparse 0.6.8
    Uninstalling torch-sparse-0.6.8:
      Successfully uninstalled torch-sparse-0.6.8
Looking in links: https://s3.eu-central-1.amazonaws.com/pytorch-geometric.com/whl

# Code challenge

In [36]:
import random
import numpy as np
import torch
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
from torch_geometric.nn import SGConv, GATConv
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T

In [37]:
dataset_pubmed = Planetoid(root="./tmp", name="Pubmed",transform=T.NormalizeFeatures())
data_pubmed = dataset_pubmed[0]

In [5]:
def compute_accuracy(model, data, mask):
  # Set the model.training attribute to False
  model.eval()
  logprob = model(data)
  y_pred = logprob[mask].max(1)[1]
  y_true=data.y[mask]
  acc = y_pred.eq(y_true).sum()/ mask.sum().float()
  return acc.item()

In [6]:
def predict(model, data):
    #acc_test = compute_accuracy(model, data, data.test_mask)
    logprob = model(data)
    y_pred = logprob[data.test_mask].max(1)[1]
    if torch.cuda.is_available():
      y_pred = y_pred.cpu().detach().numpy().reshape(-1,)
    else:
      y_pred = y_pred.numpy().reshape(-1,)
    return y_pred

In [7]:
def train(model, data, optimizer):
  # Set the model.training attribute to True
  model.train() 

  # Reset the gradients of all the variables in a model
  optimizer.zero_grad() 

  # Get the output of the network. The output is a log probability of each
  log_softmax = model(data) 

  labels = data.y # Labels of each node

  # Use only the nodes specified by the train_mask to compute the loss.
  nll_loss = F.nll_loss(log_softmax[data.train_mask], labels[data.train_mask])
  
  #Computes the gradients of all model parameters used to compute the nll_loss
  #Note: These can be listed by looking at model.parameters()
  nll_loss.backward()

  # Finally, the optimizer looks at the gradients of the parameters 
  # and updates the parameters with the goal of minimizing the loss.
  optimizer.step() 
  

In [8]:
@torch.no_grad() # Decorator to deactivate autograd functionality  
def score(model, data, test=False):
  if test:
    acc_test = compute_accuracy(model, data, data.test_mask)
    return acc_test
  else:
    acc_train = compute_accuracy(model, data, data.train_mask)
    acc_val = compute_accuracy(model, data, data.val_mask)
    return acc_train, acc_val

In [9]:
# TODO: complete your code here
class GATNet(torch.nn.Module):
  def __init__(self, data, heads_layer1, 
               heads_layer2, dropout, dropout_alphas):
    pass
  
  def forward(self, data):
    pass

In [43]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

data_pubmed= data_pubmed.to(device)

# TODO: pass in necessary parameter for GATNet
model_pubmed_gat = GATNet(data=data_pubmed, heads_layer1=8, heads_layer2=8, dropout=0.6, dropout_alphas=0.6).to(device)

optimizer = torch.optim.Adam(model_pubmed_gat.parameters(), lr=0.001, weight_decay=0.005)

In [53]:
log = 'Epoch: {:03d}, Train: {:.4f}, Val: {:.4f}' # for displaying performance during training
for epoch in range(1, 201):
    train(model_pubmed_gat, data_pubmed, optimizer)
    if epoch %10 ==0:
      train_acc, val_acc = score(model_pubmed_gat,data_pubmed)
      print(log.format(epoch, train_acc, val_acc))


Epoch: 010, Train: 0.9000, Val: 0.7160
Epoch: 020, Train: 0.9000, Val: 0.7360
Epoch: 030, Train: 0.9000, Val: 0.7480
Epoch: 040, Train: 0.9167, Val: 0.7580
Epoch: 050, Train: 0.9333, Val: 0.7680
Epoch: 060, Train: 0.9333, Val: 0.7780
Epoch: 070, Train: 0.9500, Val: 0.7940
Epoch: 080, Train: 0.9667, Val: 0.7840
Epoch: 090, Train: 0.9667, Val: 0.8000
Epoch: 100, Train: 0.9667, Val: 0.7860
Epoch: 110, Train: 0.9667, Val: 0.7940
Epoch: 120, Train: 0.9833, Val: 0.7960
Epoch: 130, Train: 0.9833, Val: 0.7960
Epoch: 140, Train: 0.9833, Val: 0.8000
Epoch: 150, Train: 1.0000, Val: 0.7920
Epoch: 160, Train: 0.9833, Val: 0.7920
Epoch: 170, Train: 1.0000, Val: 0.8000
Epoch: 180, Train: 1.0000, Val: 0.7980
Epoch: 190, Train: 0.9833, Val: 0.8040
Epoch: 200, Train: 1.0000, Val: 0.8060
Final test accuracy: 0.7840


In [45]:
# Don't display performance on test dataset until after you are happy with the performance on the validation set
# TODO change display_test_acc to True when you are ready to check the performance of the model on the hidden test set
display_test_acc = False
if display_test_acc:
  test_acc = score(model_pubmed_gat, data_pubmed, test=True)
  print("Final test accuracy: {:.4f}".format(test_acc))


Final test accuracy: 0.7780
